### Import

In [3]:
import os
import copy
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cma

from gymnasium.spaces import Dict, Box, Discrete, MultiDiscrete
from torch import nn as nn

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import CheckpointCallback

### Utils

In [4]:
def is_between(A, b):
    return A[0] < b <= A[1]

def calculate_soil_evap_rate(T, R):
    A0 = 1.0
    A1 = 0.1
    return np.float32(A0*(1 + A1*T)*(1-R*0.01))

def calculate_plant_evap_rate(T, R, light_on_off):
    A2 = 1.0
    A3 = 0.1
    A4 = 0.1
    return np.float32(A2*(1 + A3*T)*(1-R*0.01)*(1+A4*light_on_off))

### PlantAirControl Environment

In [5]:
class PlantAirControl(gym.Env):
    # Constants are hard-coded for now but can set up to read from a spreadsheet
    CHAMBER_VOLUME = 1.0
    NON_LIGHT_HEAT = 0.5
    LIGHT_HEAT = 1.0

    FAN_MAX_WATTAGE = 0.5 # kW
    HEAT_MAX_WATTAGE = 1.5 # kW
    COOL_MAX_WATTAGE = 1.5 # kW

    FAN_MAX_AIR_INOUT_RATE = 10.0 
    HEAT_MAX_WATER_TEMP_UP_RATE = 50.0
    COOL_MAX_WATER_TEMP_DOWN_RATE = 50.0

    DESIRED_TEMPS = [30.0, 40.0] # Celcius
    DESIRED_HUMIDS = [60.0, 80.0] 

    def __init__(self, render_mode=None,
                light_heat = LIGHT_HEAT,
                non_light_heat = NON_LIGHT_HEAT,
                chamber_volume = CHAMBER_VOLUME,

                fan_max_wattage = FAN_MAX_WATTAGE,
                heat_max_wattage = HEAT_MAX_WATTAGE,
                cool_max_wattage = COOL_MAX_WATTAGE,

                fan_max_air_inout_rate = FAN_MAX_AIR_INOUT_RATE,
                heat_max_water_temp_up_rate = HEAT_MAX_WATER_TEMP_UP_RATE,
                cool_max_water_temp_down_rate = COOL_MAX_WATER_TEMP_DOWN_RATE,

                desired_temps = DESIRED_TEMPS,
                desired_humids = DESIRED_HUMIDS,
                ):
        # Set up some variables
        self.chamber_volume = np.float32(chamber_volume)
        self.light_heat = np.float32(light_heat)
        self.non_light_heat = np.float32(non_light_heat)

        self.fan_max_wattage = np.float32(fan_max_wattage)
        self.heat_max_wattage = np.float32(heat_max_wattage)
        self.cool_max_wattage = np.float32(cool_max_wattage)

        self.fan_max_air_inout_rate = np.float32(fan_max_air_inout_rate)
        self.heat_max_water_temp_up_rate = np.float32(heat_max_water_temp_up_rate)
        self.cool_max_water_temp_down_rate = np.float32(cool_max_water_temp_down_rate)

        self.dersired_temps = np.float32(np.array(desired_temps))
        self.dersired_humids = np.float32(np.array(desired_humids))

        # Specify the action_space
        self.action_space = MultiDiscrete([101]*3) # e.g., fan capacity, heating component capacity, cooling component capacity
        
        # Speicfy the observation_space
        self.observation_space = Dict({"InTemp/InHumid/OutTemp/OutHumid/Energy": Box(-100, 100, shape=(5,)),
                                       "LightOnOff": Discrete(2),
                                        "Status": Discrete(2),})

    def reset(self, seed=None):
        super().reset(seed=seed) # To enable self.np_random seeding
        rng = self.np_random

        # Start state initialisation
        self.light_on_off = rng.integers(2)

        self.inside_temp = rng.uniform(low = 0, high = 70)
        self.inside_humid = rng.uniform(low = 40, high = 90)
        self.outside_temp = rng.uniform(low = - 10, high = 30)
        self.outside_humid = rng.uniform(low = 30, high = 70)

        self.energy = 0
        
        self.plant_OK = 0 # 0 means not in the box yet, 1 means OK
        
        observation = {"InTemp/InHumid/OutTemp/OutHumid/Energy": np.float32(np.array([self.inside_temp, 
                                                                          self.inside_humid, 
                                                                          self.outside_temp,
                                                                          self.outside_humid,
                                                                          self.energy])),
                        "LightOnOff": self.light_on_off,
                        "Status": self.plant_OK}

        info = {}

        return observation, info

    def step(self, action):
        # Caculate some quantities
        self.plant_evap_rate = calculate_plant_evap_rate(self.inside_temp, self.inside_humid, self.light_on_off)
        self.soil_evap_rate = calculate_soil_evap_rate(self.inside_temp, self.inside_humid)

        # Interprete action into parameters of mathematical model
        self.fan_air_inout_rate = (action[0] * self.fan_max_air_inout_rate)/100
        self.heat_water_temp_up_rate = (action[1] * self.heat_max_water_temp_up_rate)/100
        self.cool_water_temp_down_rate = (action[2] * self.cool_max_water_temp_down_rate)/100

        self.fan_wattage = (action[0] * self.fan_max_wattage)/100
        self.heat_wattage = (action[1] * self.heat_max_wattage)/100
        self.cool_wattage = (action[2] * self.cool_max_wattage)/100

        # Update observations via model for temp, humididty, VPD
        self.inside_temp += (self.non_light_heat + self.light_on_off * self.light_heat)/self.chamber_volume \
            + self.fan_air_inout_rate * (self.outside_temp - self.inside_temp) \
            + self.heat_water_temp_up_rate - self.cool_water_temp_down_rate

        self.inside_humid += (self.plant_evap_rate + self.soil_evap_rate)/self.chamber_volume \
            + self.fan_air_inout_rate * (self.outside_humid - self.inside_humid)

        self.energy += self.fan_wattage + self.heat_wattage + self.cool_wattage

        if is_between(self.dersired_temps, self.inside_temp) and is_between(self.dersired_humids, self.inside_humid):
            self.plant_OK = 1

        observation = {"InTemp/InHumid/OutTemp/OutHumid/Energy": np.float32(np.array([self.inside_temp, 
                                                                          self.inside_humid, 
                                                                          self.outside_temp,
                                                                          self.outside_humid,
                                                                          self.energy])),
                        "LightOnOff": self.light_on_off,
                        "Status": self.plant_OK}

        # Reward
        reward = self.plant_OK * 10 - self.energy

        terminated = True # episode completes after 1 step

        info = {}
            
        return observation, reward, terminated, False, info

    def plot_control(self):
        


### Testing

In [6]:
env = PlantAirControl()
episodes = 10
for episode in range(1, episodes + 1):
    obsINI, infoINI  = env.reset()
    print(obsINI)
    score = 0
    terminated = False
    truncated = False

    while not terminated or truncated:
        action = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(action)
        print(obs)
        print(f"The final reward is {reward}")

{'InTemp/InHumid/OutTemp/OutHumid/Energy': array([23.50864 , 72.57317 ,  5.914425, 68.04297 ,  0.      ],
      dtype=float32), 'LightOnOff': 1, 'Status': 0}
{'InTemp/InHumid/OutTemp/OutHumid/Energy': array([-68.443596,  52.75819 ,   5.914425,  68.04297 ,   1.74    ],
      dtype=float32), 'LightOnOff': 1, 'Status': 0}
The final reward is -1.74
{'InTemp/InHumid/OutTemp/OutHumid/Energy': array([12.633964, 54.44175 ,  8.028533, 33.358242,  0.      ],
      dtype=float32), 'LightOnOff': 1, 'Status': 0}
{'InTemp/InHumid/OutTemp/OutHumid/Energy': array([ -16.117626, -143.68614 ,    8.028533,   33.358242,    1.33    ],
      dtype=float32), 'LightOnOff': 1, 'Status': 0}
The final reward is -1.33
{'InTemp/InHumid/OutTemp/OutHumid/Energy': array([61.94746 , 45.902966, -2.848521, 39.58713 ,  0.      ],
      dtype=float32), 'LightOnOff': 0, 'Status': 0}
{'InTemp/InHumid/OutTemp/OutHumid/Energy': array([-526.13477  ,   -5.6815834,   -2.848521 ,   39.58713  ,
          2.135    ], dtype=float32),

### Training

In [7]:
log_path = os.path.join(os.getcwd(), "Logs")
save_path = os.path.join(os.getcwd(), "Saved Models")

In [8]:
 # Set up the training environment
n_envs = 8
envs = make_vec_env(PlantAirControl, 
                    n_envs=n_envs,
                    seed=42,
                    vec_env_cls=DummyVecEnv)

# Set up the training model
model = PPO("MultiInputPolicy", 
            envs, 
            verbose=0, 
            tensorboard_log= os.path.join(log_path, "Training"), 
            device = "cpu")

# Set up checkpoints for during training
checkpoint_callback = CheckpointCallback(save_freq= 1e4, 
                                            save_path=os.path.join(save_path, 'Checkpoints'),
                                            name_prefix='Checkpoint',
                                            save_replay_buffer=True,
                                            save_vecnormalize=True,
                                            verbose = 0)

# Training
model.learn(1e5, callback = checkpoint_callback, progress_bar = True)

# Save the model
model_final_path = os.path.join(save_path, 'Final') 
model.save(model_final_path)

Output()

### Deployment

In [10]:
# Load env and model
env = PlantAirControl()
model = PPO.load(save_path,env,tensorboard_log=log_path)

PermissionError: [Errno 13] Permission denied: 'd:\\GitHub\\Maths4Food\\Saved Models'

In [ ]:
episodes = 5
for episode in range(1, episodes + 1):
    obsINI, infoINI  = env.reset()
    score = 0
    terminated = False
    truncated = False

    while not terminated or truncated:
        action, _  = model.predict(obsINI, deterministic = False)
        obs, reward, terminated, truncated, info = env.step(action)
        score += reward
        print(f"{score}")